# 데이콘 운동동작분류 - 2등 분석 (21.03.23)

###  1D-CNN에 global-average-pooling 사용
### 타겟 변수가 굉장히 불균형하기 때문에 Stratified KFold, K=10

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')

load data

In [4]:
train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')
pd.options.display.max_columns=50

FileNotFoundError: ignored

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
train=pd.read_csv('/content/drive/MyDrive/train_features.csv')
train_labels=pd.read_csv('/content/drive/MyDrive/train_labels.csv')
test=pd.read_csv('/content/drive/MyDrive/test_features.csv')

submission=pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

pd.options.display.max_columns=50

Feature engneering
가속도, 자이로, (자이로-가속도) 센서값을 에너지로 표현

In [85]:
# 가속도
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

# 자이로
train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

# (자이로-가속도)
train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

id별 데이터는 0.02초마다 측정된 값들이기 때문에 이전 시간 대비 변화량 적용

In [86]:
dt=0.02 # 0.02초
def jerk_signal(signal): # 이전 시간 대비 변화량 = (현재값-이전값)/0.02 , (시그널의 길이-1)만큼 반복 ex) 5개 -> 4번 반복
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [87]:
train_dt=[]
# id에서 중복값 제거하고(id는 3125개) 진행표시바(중간중간 잘 생성이 되었나 확인 목적) 생성
for i in tqdm(train['id'].unique()): # for i in train['id'].unique() -> 3125
    # train['id']=i인 것의 행 반환 ex) id=1인 것의 acc_x : 000, acc_y : 000, ..., gy_z : 000
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        # np.insert():주어진 축에 대해 주어진 인덱스 앞에 값을 삽입
        # values의 첫번째 위치(index=0)에 값 0 삽입
        values=np.insert(values,0,0)
        # temp에서 v열(2열~8열) (_dt붙이고) : (acc_x_dt, ..., gy_z_dt)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████| 3125/3125 [01:00<00:00, 51.38it/s]


In [88]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|██████████| 782/782 [00:14<00:00, 54.08it/s]


가속도, 자이로 센서값들을 푸리에 변환
푸리에 변환(Fourier transform, FT)은 시간이나 공간에 대한 함수를 시간 또는 공간 주파수 성분으로 분해하는 변환을 말한다. 

In [89]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    # scipy.fftpack.fft ()를 사용하여 FFT(fast Fourier transforms)를 계산
    complex_f_signal= fftpack.fft(t_signal)
    # fft() 명령어를 수행한 결과를 살펴보면 파형을 실수부와 허수부로 나뉘는 역할만을 하게 됩니다.
    # 주파수 해석의 목적은 각 주파수 별로 크기 (magnitude)가 어느 정도인지 알아내는 것이기 때문에
    # 절대값 함수 abs()를 이용해서 크기를 연산할 수 있습니다.
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal


In [90]:
train=pd.concat(train_dt)

In [12]:
train.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.146962,12.465436,12.427938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,1.200703,12.913284,12.865692,4.080495,-0.980114,-1.699854,44.735403,-429.504677,337.444793,2.687024,22.392358,21.887693
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,1.217403,13.725729,13.692643,0.845632,0.192961,-3.546937,-196.018888,-249.173073,-4.620631,0.835012,40.622253,41.347563
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,1.209981,15.374021,15.314907,-0.575711,-1.762585,1.908626,316.513181,-473.763910,-121.719195,-0.371100,82.414636,81.113199
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,1.211254,16.074363,16.017964,0.389598,2.130453,-0.365665,78.686055,-215.435892,-25.361098,0.063656,35.017060,35.152822


In [91]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████| 3125/3125 [00:16<00:00, 188.88it/s]


In [92]:
test=pd.concat(test_dt)

In [93]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|██████████| 782/782 [00:02<00:00, 304.33it/s]


Standard scaling 적용

In [94]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [95]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
# 평균 0 , 분산 1로 조정
scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)
# train_sc, test_sc

In [18]:
train_sc.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,27.356382,8.807207,19.465910,0.376992,0.869226,0.150423,0.495681,-0.272719,-0.276391,0.000027,0.000298,-0.000433,0.000347,0.000373,0.000273,0.000101,0.001505,0.001501
1,0,1,-0.054866,0.833464,0.820412,-0.282128,-0.093560,0.011266,0.742974,-0.236152,-0.240632,0.416836,-0.118821,-0.255054,0.032738,-0.349095,0.377085,0.564992,0.166566,0.162871
2,0,2,0.024046,0.315921,0.081086,-0.182551,-0.053585,-0.003708,0.819822,-0.169815,-0.173080,0.086405,0.023750,-0.531727,-0.141582,-0.202368,-0.004887,0.175645,0.300944,0.306341
3,0,3,0.065632,0.117634,-0.040874,-0.194863,0.154242,0.005408,0.785669,-0.035229,-0.040560,-0.058780,-0.213920,0.285459,0.229520,-0.385106,-0.135647,-0.077915,0.609008,0.599518
4,0,4,0.151477,0.300751,0.317742,-0.350724,0.494539,0.154354,0.791528,0.021954,0.016872,0.039823,0.259227,-0.055206,0.057320,-0.174917,-0.028047,0.013483,0.259626,0.260669


모델링
CNN, LSTM, CNN+LSTM 등 여러 구조 적용해보다가 CNN에서 Flatten 없이 Global average pooling 한 구조가 가장 성능이 좋아 채택했습니다.

In [97]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras import backend as K 
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
import keras

In [96]:
!pip install tensorflow_addons

     |████████████████████████████████| 706kB 6.1MB/s 


In [98]:
X=np.array(train_sc.iloc[:,2:]).reshape(3125, 600, -1)
X.shape

(3125, 600, 18)

In [99]:
test_x=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
test_x.shape

(782, 600, 18)

In [100]:
y = train_labels['label'].values
# tf.keras.utils.to_categorical : Converts a class vector (integers) to binary class matrix.
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [101]:
# cnn model (tensorflow)
def cnn_model(input_shape, classes):
    seed(2021)
    tf.random.set_seed(2021)
    
    input_layer = keras.layers.Input(input_shape)
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model

In [29]:
model_ex = cnn_model((600,18),61)
model_ex.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 600, 18)]         0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 600, 128)          20864     
_________________________________________________________________
batch_normalization_6 (Batch (None, 600, 128)          512       
_________________________________________________________________
activation_6 (Activation)    (None, 600, 128)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 600, 128)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 600, 256)          196864    
_________________________________________________________________
batch_normalization_7 (Batch (None, 600, 256)          1024

pytorch

In [19]:
# cnn model in pytorch
X=np.array(train_sc.iloc[:,2:]).reshape(3125, -1, 600)
X.shape

(3125, 18, 600)

In [20]:
test_x=np.array(test_sc.iloc[:,2:]).reshape(782, -1, 600)
test_x.shape

(782, 18, 600)

In [34]:
train_labels.label.unique()

array([37, 26,  3, 39, 28,  6, 60, 35, 34, 30, 59, 15, 40, 50, 48,  8, 54,
       16,  4, 58, 21, 18, 51, 25, 24, 17, 49, 47, 27, 36,  2, 38, 14, 10,
       55, 57,  0,  1, 46, 43, 52, 53,  5, 56, 23, 44, 42, 19, 32, 45, 29,
       31, 11,  7, 41, 12,  9, 20, 13, 22, 33])

In [35]:
y = train_labels['label'].values

# pytorch

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

y = to_categorical(train_labels['label'],61)

In [36]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=uint8)

In [160]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class cnn_model(nn.Module):
  def __init__(self):
    super(cnn_model,self).__init__()
    seed(2021)
    torch.manual_seed(2021)
    
  
    # 합성층 1
    self.layer1 = nn.Sequential(
              nn.Conv1d(in_channels = 18, out_channels = 128 , kernel_size = 9, padding=4),
              nn.BatchNorm1d(128),
              nn.ReLU(),
              nn.Dropout(0.3))
    self.layer2 = nn.Sequential(
              nn.Conv1d(in_channels = 128, out_channels = 256 , kernel_size = 7, padding=3),
              nn.BatchNorm1d(256),
              nn.ReLU(),
              nn.Dropout(0.4))
    self.layer3 = nn.Sequential(
              nn.Conv1d(in_channels = 256, out_channels = 128 , kernel_size = 3, padding=1),
              nn.BatchNorm1d(128),
              nn.ReLU(),
              nn.Dropout(0.5))
    
    self.fc = nn.Linear(128,61)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = nn.AdaptiveAvgPool2d(out)
    out = self.fc(out)
    out = F.softmax(out)
    return out


In [161]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = cnn_model().to(device)

In [162]:
from torchsummary import summary
summary(model, input_size=(18,600))

TypeError: ignored

In [127]:
from torchvision import models
model = models.vgg16()
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [133]:
from torchvision import models
from torchsummary import summary

vgg = models.vgg16()
summary(vgg, (2,600,18))

RuntimeError: ignored

In [77]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

GAP의 목적은 feature를 1차원 벡터로 만들기 위함
GAP는 CNN + FC(Fully Connected) Layer에서 classifier인 FC Layer를 없애기 위한 방법으로 도입
어떤 사이즈의 입력이 들어와도 상관없고 학습, 과적합 유리
GAP 연산 결과는 1차원 벡터
경우에 따라서 FC layer와 같이 사용 되기도 합니다. FC layer에 전달하기 전에 GAP를 이용하여 차원을 줄여서 벡터로 만든 다음에 FC layer로 전달 하면 FC Layer에서 쉽게 사이즈를 맞출 수 있기 때문입니다.

10-fold StratifiedKFold (분류문제에서 label의 분포가 각 class별로 불균형을 이룰 때 유용)

In [81]:
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
es =EarlyStopping(monitor='val_loss', patience=8, mode='min')

accuracy = []
losss=[]
models=[]

for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
    mc = ModelCheckpoint(f'./model_kf/cv_study{i + 1}.h5',save_best_only=True, verbose=0, monitor = 'val_loss', mode = 'min', save_weights_only=True)
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)
    model = model((nn.Module)
    history = model.fit(X[train], y[train], epochs = 100, 
                        validation_data= (X[validation], y[validation]), verbose=1,batch_size=64,callbacks=[es,mc,reLR])
    model.load_weights(f'./model_kf/cv_study{i + 1}.h5')
    
    k_accuracy = '%.4f' % (model.evaluate(X[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(X[validation], y[validation])[0])
    
    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

SyntaxError: ignored

pytorch end

In [102]:
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
es =EarlyStopping(monitor='val_loss', patience=8, mode='min')

accuracy = []
losss=[]
models=[]

for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
    mc = ModelCheckpoint(f'./model_kf/cv_study{i + 1}.h5',save_best_only=True, verbose=0, monitor = 'val_loss', mode = 'min', save_weights_only=True)
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)
    model = cnn_model((600,18),61)
    history = model.fit(X[train], y[train], epochs = 100, validation_data= (X[validation], y[validation]), 
                        verbose=1,batch_size=64,callbacks=[es,mc,reLR])
    model.load_weights(f'./model_kf/cv_study{i + 1}.h5')
    
    k_accuracy = '%.4f' % (model.evaluate(X[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(X[validation], y[validation])[0])
    
    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

--------------------Fold_1--------------------
Epoch 1/100
44/44 [==============================] - 79s 2s/step - loss: 3.3854 - accuracy: 0.3490 - val_loss: 3.1167 - val_accuracy: 0.3099


OSError: ignored

성능 확인 및 제출

In [ ]:
print(sum([float(i) for i in accuracy])/10)
print()
print(sum([float(i) for i in losss])/10)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-44-709c0f85d52a>", line 1, in <module>
    print(sum([float(i) for i in accuracy])/10)
NameError: name 'accuracy' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\leesa\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\leesa\anaconda3\li

NameError: name 'accuracy' is not defined

In [ ]:
test_X=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
test_X.shape

In [ ]:
preds = []
for model in models:
    pred = model.predict(test_X)
    preds.append(pred)
pred = np.mean(preds, axis=0)
pred

In [ ]:
submission=pd.read_csv('./data/sample_submission.csv')
submission.iloc[:,1:]=pred
submission
submission.to_csv('sub_kfold_stratified_10_adam_fft_0.5.csv',index=False)